<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Michael/592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [2]:
controlData = np.loadtxt('Robot Log/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float)
print(np.max(controlData[:,0]))
controlData[:,0] /= 15

15.0


In [3]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:799])
ImagesTrain = ImagesTrain.unsqueeze(1)
steerDataTrain = torch.tensor(controlData[0:799, 0])
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[800:999])
ImagesTest = ImagesTest.unsqueeze(1)
steerDataTest = torch.tensor(controlData[800:999, 0])
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:799, 1])
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[800:999, 1])
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

In [4]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=128, shuffle=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=128, shuffle=False)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=128, shuffle=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=128, shuffle=False)

In [5]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

class LRthrottle(torch.nn.Module):
    def __init__(self):
        super(LRthrottle, self).__init__()
        self.conv2d1 = torch.nn.Conv2d(1, 1, 4, 1)
        self.linear1 = torch.nn.Linear(157, 56) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(56, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        transformed_x1 = self.conv2d1(x)
        transformed_x1 = transformed_x1.view(-1, 157)
        transformed_x2 = self.linear1(transformed_x1)
        transformed_x3 = self.linear2(transformed_x2)
        transformed_x4 = self.linear3(transformed_x3)
        transformed_x5 = self.linear4(transformed_x4)
        return transformed_x5

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.L1Loss()
throttleNet = LRthrottle()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=0.01)

In [6]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(30):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        predicted_output = steerNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 26451.645117617078 and test loss 16140.265625
Epoch 1 finished with train loss 7924.398603166853 and test loss 3602.2913818359375
Epoch 2 finished with train loss 1048.6529643195015 and test loss 688.9280395507812
Epoch 3 finished with train loss 299.36566182545255 and test loss 768.0343933105469
Epoch 4 finished with train loss 462.6896795545305 and test loss 37.53192710876465
Epoch 5 finished with train loss 205.4369000026158 and test loss 72.23158645629883
Epoch 6 finished with train loss 45.04077570778983 and test loss 99.2734260559082
Epoch 7 finished with train loss 42.586144753864836 and test loss 7.044230699539185
Epoch 8 finished with train loss 22.33863503592355 and test loss 8.974209308624268
Epoch 9 finished with train loss 4.5337474175861905 and test loss 9.52082109451294
Epoch 10 finished with train loss 5.073021941951343 and test loss 0.13356386125087738
Epoch 11 finished with train loss 2.024192201239722 and test loss 1.290830135345459
E

In [7]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(40):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([40576, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([9827, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([22507, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(

Epoch 0 finished with train loss 0.6781797196183886 and test loss 0.5292550474405289
Epoch 1 finished with train loss 0.38422764199120657 and test loss 0.3550485074520111
Epoch 2 finished with train loss 0.2930607795715332 and test loss 0.27944520115852356
Epoch 3 finished with train loss 0.2712752946785518 and test loss 0.2523506134748459
Epoch 4 finished with train loss 0.2557558502469744 and test loss 0.2619936019182205
Epoch 5 finished with train loss 0.24054477044514247 and test loss 0.25063347071409225
Epoch 6 finished with train loss 0.23764079809188843 and test loss 0.24722076952457428
Epoch 7 finished with train loss 0.23955304069178446 and test loss 0.25239768624305725
Epoch 8 finished with train loss 0.23808166810444423 and test loss 0.24835597723722458
Epoch 9 finished with train loss 0.23374055538858687 and test loss 0.2513800114393234
Epoch 10 finished with train loss 0.23638020668710982 and test loss 0.24895911663770676
Epoch 11 finished with train loss 0.236323859010423

In [8]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)
Images = torch.tensor(Images)
Images = Images.unsqueeze(1)
Images = Images.unsqueeze(1)

for i in np.linspace(1, 300, 100).astype(int):
  predicted_output_steer = steerNet(Images[i].float())
  predicted_output_throttle = throttleNet(Images.float()[i])
  correct_steer = controlData[i,0]
  correct_throttle = controlData[i,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle[i]))

correct steer: 0.0 predicted steer: tensor([[0.0322]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.3965], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.1120]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.3979], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.0160]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.3963], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.1077]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.3947], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.0230]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.3958], grad_fn=<SelectBackward0>)
correct steer: 0.0 predicted steer: tensor([[0.0746]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([0.3962], grad_fn=<SelectBackward0